<a href="https://colab.research.google.com/github/Centrattic/AI4ALL_2021/blob/main/Riya_P4_Data_Exploration_and_Scikit_Learn_Basics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Exploration and Scikit-Learn

In this final Python lesson, we'll use everything we've learned so far and work with some real datasets. We'll also finally use the Scikit-Learn library (called `sklearn` in Python) to do some basic machine learning on some sample data.

## Some important imports

In Jupyter Notebooks, we can import the libraries we need once, and then use the libraries on all future code blocks without needing to re-import everything. 

Run the following code block:

In [1]:
%matplotlib inline
import sklearn
import pandas
import numpy as np
import matplotlib.pyplot as plt

## Get that dataset!

Before we can start to do machine learning, we need to have data to train our systems on. If you remember in the "Introduction to Machine Learning" lesson, datasets aren't always easy to come by, and many are privately collected and owned. Very often, if you want to work on a new machine learning application, you may have to figure out how to collect your own data.

However, there are many great services that have open and free datasets that you can use, like [Kaggle](https://www.kaggle.com/). For this lesson, we'll use a modified dataset from Kaggle that contains Spotify information on the top songs of 2017.

## Grabbing datasets from GitHub

We've actually stored the dataset in a different website, called [GitHub](https://github.com/), so that we can easily grab the data for this lesson.

Be sure to run each of the code blocks below. They're not written in Python -- they're written in a different language called Bash, which is typically used on the command line and can manipulate things like files on your computer.

You don't have to worry about what each of these code blocks does, just that they help us pull data from GitHub so that we have access to it within this Notebook. (Almost like how we need to import libraries before we can start to use them.)

In [ ]:
# The following is not a Python command, but a bash command
# It looks at the files and folders in this current directory
! ls

In [ ]:
# This checks if the directory called "spotify-data-ai4all" exists, and if so, remove it
! if [ -d "spotify-data-ai4all" ]; then rm -r spotify-data-ai4all; fi
! ls

In [ ]:
# This creates a new directory based on a repo from my GitHub
! git clone https://github.com/mleszczy/spotify-data-ai4all

In [ ]:
# This is for us to check that the directory was successfully downloaded
! ls

# Spotify's Top Songs of 2017

Let's say that you were interested in music and wanted to create a machine learning system to suggest a new song for someone to listen to. How would we create this?

Well first, we would need to have some data on what music people like. Once we have that data, we might start to think about the different qualities of a song, so you can recommend new songs based on similar qualities from songs a person has listened to.

The Spotify Top Songs of 2017 dataset contains exactly that information, so let's see how we would start to use it.

## Working with Pandas

Before we jump into machine learning, let's practice looking at datasets and exploring them. We'll use a library called `pandas` to help us do this.

We've already imported `pandas` earlier in the Notebook, so we can use it in the following code blocks. If you did not run the first code block of this lesson, the following code will raise an error.

In [ ]:
# This is the file name of the Spotify dataset
# It's a "CSV" file, which stands for "comma-separated values file"
# Really, it's just a long text file with a lot of information separated by commas
filename = "spotify-data-ai4all/spotify.csv"

# We will use a function built into pandas to read our csv data
data = pandas.read_csv(filename)

# Finally, we'll print the data to take a look at it
print(data.head())

The print-out of our data above looks... not too informative. This is mainly because Jupyter Notebooks doesn't format it nicely for us when we directly print all of the data. Instead, we'll have to use some functions in `pandas` to explore the data ourselves.

By the way, if you want to see the data in this dataset as a table, you can [click here to see it displayed in GitHub](https://github.com/mleszczy/spotify-data-ai4all/blob/master/spotify.csv). We'll also write some code so that we can visualize our data in table form.

There are 36 rows in this dataset and 17 columns, where each row is a single song and (almost) each column is a **feature** of the song. In terms of dataset size, this is a very small dataset for example purposes -- some datasets may have thousands or even millions of rows.

## Exploring some features

Recall that our data in datasets is described by a set of properties called features.

For song data, some features you might expect are tempo, time signature, and name.

Let's print all of the names of the songs:

In [ ]:
# Notice that we didn't just say "name"
# Like with accessing values in dictionaries, the specific key value in this data set is "artist"
print(data["song_title"])

The above code prints the song titles for all 2017 songs in our dataset. However, if we wanted to recommend a new song to someone, it probably wouldn't be enough to just recommend them based on song title. We'll want to make use of multiple features, and it may help us to visualize them to find patterns in the data.

How do we know what all of the features in our dataset are? One way is to look directly at the CSV file, because these are usually the column headers that are on the first line of the file. (We could also look at the column headings in GitHub since it displays all of the data in a table nicely for us.) Since we're trying to work with our Python code as much as possible, let's see how we would do it in Python:

In [ ]:
filename = "spotify-data-ai4all/spotify.csv"
raw_data = pandas.read_csv(filename)

# Get the values of the columns (the features) and print them out
features = raw_data.columns.values
print(features)

We see from the printout different features of our songs, such as tempo, acoustiness (how many acoustic sounds, like an acoustic guitar, are in a song), title, and artist. The first column is actually used to index the songs (so we can refer to them by number), and that's why the column title shows as "Unnamed: 0". This particular column is not a feature of our data.

For now, let's work with just a small amount of our dataset -- four features (tempo, acousticness, song title, and artist) and 20 songs.

In [ ]:
filename = "spotify-data-ai4all/spotify.csv"
raw_data = pandas.read_csv(filename)

# In pandas, we put our data into data frames so that we can manipulate them
# This takes two parameters: raw_data specifies that we're using the music data from above
# columns specifies what columns of that data we want to keep
data = pandas.DataFrame(raw_data, columns=['tempo', 'acousticness', 'song_title', 'artist'])

# Remember slicing? We use that to take just the first 20 songs in our dataset
data = data[:20]

# This will display our data in a nice table
# Typically we need to use print(), but our variable 'data' is a pandas DataFrame which has special display properties
data

There! Now we have something that's more manageable.

## Comparing features of songs

Say we know someone likes the song "Fireflies" by Owl City, and we want to recommend them another song based off of similarity to "Fireflies".

Perhaps we think that the tempo (or speed) of the song might be helpful, so we'll look for songs that are similar in tempo to "Fireflies" (which is tempo = 90.002 beats per minute).

In [ ]:
# Let's keep track of the Fireflies tempo
fireflies_tempo = 90.002

# Let's create a new column in our dataset
# The value of this column is the difference in tempo from that song and the tempo of "Fireflies"
# The function abs() takes the absolute value, because we just need to know the difference in tempo,
# but not if that tempo is faster or slower than "Fireflies"
data['tempo_difference'] = abs(fireflies_tempo - data.tempo)
data

### Visualizing the tempo difference

We can look at the column `tempo_difference` in our table above to see which songs have similar tempos to "Fireflies". Another way to present this information is to visualize it in a graph:

In [ ]:
# Remember that we've imported matplotlib.pyplot as plt at the start of the Notebook
# Here we will use pyplot to do our visualization
plt.figure(figsize=(8,8))
plt.scatter(data['tempo_difference'], data['song_title'])
plt.xlabel('Difference in Tempo')
plt.show()

Take a moment to look at the graph and see what's going on. The y-axis of the graph is _discrete_ -- each y-value refers to a specific song. The x-axis of the graph is the tempo difference. 

Points that are closer to the left have smaller differences in tempo, whereas points that are closer to the right have larger differences in tempo.

Looking at the graph above, which song has the largest difference in tempo from "Fireflies"?

### Finding the acousticness difference

Let's do the same for acousticness, which we might think is another good measure of song similarity.

The acousticness of a song is measured from 0 to 1, with 0 being no acousticness (all electronic sounds) and 1 being all acoustic (imagine an acoustic guitar on its own). The acoustiness of "Fireflies" is 0.02940.

In [ ]:
# Add new column for acoustic difference
data['acoustic_difference'] = abs(0.02940 - data.acousticness)
data

In [ ]:
# Visualize it
plt.figure(figsize=(8,8))
plt.scatter(data['acoustic_difference'], data['song_title'])
plt.xlabel('Difference in Acousticness')
plt.show()

What song is most different in acousticness to "Fireflies"?

## Recommendation based on similarity

Now we want to recommend a song based on its similarity to "Fireflies", in terms of both tempo and acousticness. Looking at the two graphs above, which song might you say is closest to "Fireflies" for both features?

Take a moment to look at both graphs and try to decide!



---

One possible recommendation is the song "All I Have to Give", which is similar in both tempo and acousticness. Let's listen to a bit of the two songs to see if we think that's accurate:

> Fireflies: https://www.youtube.com/watch?v=psuRGfAaju4&ab_channel=OwlCityVEVO

> All I Have to Give: https://www.youtube.com/watch?v=pj6FCKm8dhM&ab_channel=BackstreetBoysVEVO

Though they sound very different from one another, you can indeed say that the features like tempo are similar. However, upon listening to the songs themselves, you might come to the conclusion that they don't _really_ sound similar. Perhaps other features of songs, like genre, may be better features to recommend off of. It's an open question! 

Part of the work when developing a recommendation system like this is to see which features give better recommendations, and to select what the best features might be. We might know that "Fireflies" and "All I Have to Give" are similar songs in terms of tempo and acoustiness, but this might not mean that they're similar with regards to other features.

### Quantifying similarity with multiple features

Before we move on to using Scikit-Learn, let's try to quantify the similarity between songs using both tempo and acousticness together. Previously, we had calculated the tempo difference and the acousticness difference separately, and can compare songs based on one feature or the other. 

When we tried to find the most similar song using both features, we mainly approached it by visually inspecting the two graphs.

But how can we calculate similarity with respect to both features, in a quantifiable way?

First, let's see if we can visualize our 20 songs on the same graph, with both tempo and acoustiness features represented:

In [ ]:
# The x-axis is tempo
# The y-axis is acousticness
plt.scatter(data['tempo_difference'], data['acoustic_difference'])
plt.xlabel('Tempo Difference')
plt.ylabel('Acoustic Difference')

# Set the figure size of the plot so it's not too small to read
plt.rcParams["figure.figsize"] = 20, 20

# This loop goes through each point on the graph and labels it with the song title
for index, row in data.iterrows():
    plt.annotate(row['song_title'], xy=(row['tempo_difference'], row['acoustic_difference']))

From this visualization, where we look at both features at once, it's a lot clearer that "All I Have to Give" is very similar to "Fireflies" in terms of both acousticness and tempo difference, whereas "This I Promise You" is very different.



---

On the visualization above, we might realize that we can quantify 'similarity' as the distance between two points. The shorter the distance from "Fireflies", the more similar the song represented by that point is. We can make use of this intuition to numerically compute similarity across both features.

How do we compute distance again? We use the distance formula!

$$ distance = \sqrt{(y_2 - y_1)^{2} + (x_2 - x_1)^{2}} $$

In words, we take the squared difference of both features, add them, and then take the square root.

Let's say $ (x_2, y_2) $ is our point representing "Fireflies", which is at $(0,0)$. To find the distance to the point representing "All I Have to Give", we substitute for $x_1$ and $y_1$ the values of "All I Have to Give", which are 5.979 and 0.01000.

Written out:

$$ distance = \sqrt{(0.027 - 0)^{2} + (5.979 - 0)^{2}} $$
$$ distance = 5.979 $$ 

We can write this out in code and create a new column based on "distance" from Fireflies":

In [ ]:
fireflies_tempo = 90.002
fireflies_acousticness = 0.02940
data["distance"] = np.sqrt((data.acousticness - fireflies_acousticness) ** 2 + (data.tempo - fireflies_tempo) ** 2)

# Let's sort the dataset in terms of distance
sorted_dataset_distance = data.sort_values('distance')

sorted_dataset_distance

Now we have a column that tells us numerically which song is closest to "Fireflies" in terms of both tempo and acousticness!

### Normalizing our data

But wait a minute... if we take a close look at the `tempo_difference` and `distance` columns, we will notice that the two are very similar. This makes sense, because while tempo values range anywhere from 0 to 161 in our dataset, acousticness only goes from 0 to 1. The `tempo_difference` completely overshadows the `acoustic_difference` when we calculate `distance`.

In data science and machine learning, it's very common for different features to work on completely different scales. This means that we cannot compare two features directly against one another, without first **normalizing** the data to the same scale across all features. Typically, normalizing our data means scaling the values of our features to a range between 0 and 1.

We won't cover normalizing in this lesson, but it's good to be aware that this is a typical process that we perform on our data before using it to train a machine learning system.